# Реляционная база данных
Команда образовательной платформы Х собрала информацию для A/B-тестирования нового экрана оплаты услуг на сайте и на её основе создала реляционную базу данных, управляемую системой Clickhouse ([источник](https://karpov.courses/analytics)). Необходимо определить количество усердных студентов, сохраненных в этой базе данных, и рассчитать различные продуктовые метрики для контрольной группы A и пилотной группы B.
## Описание данных
<div style="text-align: center;">
<b>peas</b>
</div>

| Название атрибута | Тип данных | Описание                  |
|-------------------|--------------|------------------------------------|
| st_id             | int          | ID студента                          |
| timest            | timestamp    | Время решения задачи              |
| correct           | bool         | Правильно ли решена задача?      |
| subject           | text         | Курс, из которого взята задача |


<div style="text-align: center;">
<b>studs</b>
</div>

| Название атрибута | Тип данных | Описание                  |
|-------------------|--------------|------------------------------------|
| st_id             | int          | ID студента                          |
| test_grp          | text         | Группа, к которой отнесен студент       |

<div style="text-align: center;">
<b>final_project_check</b>
</div>

| Название атрибута | Тип данных | Описание                  |
|-------------------|--------------|------------------------------------|
| st_id             | int          | ID студента                          |
| sale_time         | timestamp    | Время покупки курса             |
| money             | int          | Цена, по которой приобрели курс   |
| subject           | text         | Название курса                      |

**NOTE:** Платформа предоставляет возможность проходить курсы по модели trial, в рамках которой пользователь может бесплатно получить доступ к 30 заданиям в день. Для снятия этого ограничения необходимо приобрести полный доступ к конкретному курсу.

## Импорт библиотек
Для отправки SQL-запросов к базе данных и обработки её откликов будет использована библиотека pandahouse.

In [1]:
import pandahouse as ph
import pandas as pd
import numpy as np

## Параметры подключения к базе данных

In [2]:
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'}

## Усердные студенты
Пусть усердный студент – это такой пользователь платформы, что хотя бы один раз правильно решил 20 и более заданий за месяц. Напишем запрос к базе данных, который выведет количество таких студентов.

Прежде всего, определим, за какой период времени собраны данные.

In [3]:
# Запрос вида MIN(timest) as min_date приводит к ошибке pandahouse.
# Поэтому считываем время как строку.
query = '''
SELECT 
    CAST(MIN(timest) as VARCHAR) as min_date, 
    CAST(MAX(timest) as VARCHAR) as max_date
FROM {db}.peas
'''
df = ph.read_clickhouse(query, connection=connection_default)
df

,min_date,max_date
0,2021-10-30 13:00:00.000,2021-10-31 17:29:52.000


Видно, что в данных содержатся записи только за 2 дня октября 2021-го. 
Тем не менее, при составлении запроса для вывода числа усердных студентов учтём, что подобные **peas** таблицы гипотетически могут содержать информацию, собранную за несколько месяцев и даже лет.

In [4]:
query = '''
-- Вывод числа усердных студентов за каждый месяц, содержащийся в данных. 
SELECT
    mod_date,
    count(st_id) as diligent_students
FROM
    (
    SELECT
        toStartOfMonth(timest) as mod_date,
        st_id
    FROM peas
    -- Группировка записей в таблице по месяцам и id студентов.
    GROUP BY mod_date, st_id
    /* Подсчет количества заданий, выполненных студентом за текущий месяц.
    Если студент решил больше 20 заданий за текущий месяц, сохраняем его id и
    соответствующую дату в результирующей таблице. */
    HAVING sum(correct) >= 20
    )
GROUP BY mod_date
'''
df = ph.read_clickhouse(query, connection=connection_default)
df

,mod_date,diligent_students
0,2021-10-01,136


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   mod_date           1 non-null      datetime64[ns]
 1   diligent_students  1 non-null      uint64        
dtypes: datetime64[ns](1), uint64(1)
memory usage: 144.0 bytes


## Расчет продуктовых метрик 
С помощью *одного запроса* к базе данных выведем следующую информацию о группах пользователей:
* средний доход с пользователя;
* средний доход с платящего пользователя;
* средний доход с активного пользователя; 
* коэффициент конверсии пользователей в покупку; 
* коэффициент конверсии активных пользователей в покупку;
* коэффициент конверсии пользователей, проявлявших активность в курсе математики, в покупку этого курса.

**Notes**:
<br>*Активным* считается пользователь, за все время решивший правильно более 10 любых задач.
<br>*Активным по математике* считается пользователь, который за все время правильно решил две или более задачи в курсе математики.

Прежде всего, опишем, как будут рассчитываться метрики, а также введем их обозначения для применения в итоговой таблице.
* Средний доход с пользователя (**ARPU**) - отношение дохода, полученного со всех пользователей, к числу всех пользователей.
* Средний доход с платящего пользователя (**ARPPU**) - отношение дохода, полученного со всех пользователей, к числу пользователей, совершивших хотя бы одну покупку.
* Средний доход с активного пользователя (**ARPАU**) - отношение дохода, полученного со всех пользователей, к числу активных пользователей.
* Коэффициент конверсии пользователей в покупку (**CR**) - отношение числа пользователей, совершивших покупку, к общему числу пользователей.
* Коэффициент конверсии активных пользователей в покупку (**CR_active**) - отношение числа активных пользователей, купивших какой-либо курс, к общему числу активных пользователей.
* Коэффициент конверсии пользователей, проявлявших активность в курсе математики, в покупку этого курса (**CR_math_active**) - отношение числа пользователей, верно решивших за всё время более 2 задач по математике и купивших курс по ней, к общему числу пользователей, проявлявших активность в математике.

Предположим, что не все пользователи покупали курсы / решали задачки. Тогда самый полный список *st_id* хранится в таблице **studs**; именно она и будет использоваться в качестве каркаса для вычислений.

Перейдем к написанию запроса.

In [6]:
query = (
'''
SELECT
    test_grp,
    sum(money_spent) / count(*) as ARPU,
    sum(money_spent) / sum(made_purchase) as ARPPU,
    sum(money_spent) / sum(active) as ARPAU,
    sum(made_purchase) / count(*) as CR,
    sum(active_made_purchase) / sum(active) as CR_active,
    sum(math_active_made_purchase) / sum(math_active) as CR_math_active
FROM
    (
    SELECT 
        a.test_grp as test_grp,
        a.st_id as st_id,
        b.money_spent as money_spent,
        -- if(X, Y, Z) = CASE WHEN X THEN Y ELSE Z END
        if(b.st_id = '', 0, 1) as made_purchase,
        if(c.st_id = '', 0, 1) as math_made_purchase,
        if(d.st_id = '', 0, 1) as active,
        if(e_.st_id = '', 0, 1) as math_active,
        if(made_purchase + active != 2, 0, 1) as active_made_purchase,
        if(math_made_purchase + math_active != 2, 0, 1) as math_active_made_purchase
    FROM studs as a
    --
    LEFT JOIN
        (
        -- Таблица со столбцами:
        -- ID студента | полный размер потраченных им денежных средств.
        SELECT
            st_id,
            sum(money) as money_spent
        FROM final_project_check
        GROUP BY st_id
        ) as b
    ON a.st_id = b.st_id
    --
    LEFT JOIN
        (
        -- Колонка с ID студентов, купивших курсы по математике.
        SELECT st_id
        FROM final_project_check
        WHERE subject = 'Math'
        GROUP BY st_id
        ) as c
    ON a.st_id = c.st_id
    --
    LEFT JOIN
        (
        -- Колонка с ID активных студентов.
        SELECT st_id
        FROM peas
        GROUP BY st_id
        HAVING sum(correct) > 10    
        ) as d
    ON a.st_id = d.st_id
    --
    LEFT JOIN
        (
        -- Колонка с ID студентов, проявивших активность в курсе математики.
        SELECT st_id
        FROM peas
        WHERE subject = 'Math'
        GROUP BY st_id
        HAVING sum(correct) >= 2
        ) as e_
    ON a.st_id = e_.st_id
    )
GROUP BY test_grp
'''
)

df = (ph.read_clickhouse(query, connection=connection_default))
df.round(3)

,test_grp,ARPU,ARPPU,ARPAU,CR,CR_active,CR_math_active
0,control,4540.984,92333.333,10905.512,0.049,0.11,0.061
1,pilot,11508.475,106093.750,35364.583,0.108,0.26,0.095


В подзапросе была создана таблица следующего вида:
| Группа | ID студента | Сколько денег потратил | Покупал курсы? | Купил курс по математике? | Был активным? | Был активным в математике? | Был активным и покупал курсы? | Был активным в математике и купил курс по ней? | 
|--------|-------------|------------------------|----------------|---------------------------|---------------|----------------------------|-----------------------------------|----------------|

И далее уже на её основе выполнялся расчет финальной таблицы с метриками тестовых групп. 